### 1. Import Packages

In [1]:
pip install -i https://test.pypi.org/simple/ my_krml_ratana

Looking in indexes: https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from joblib import dump

### 2. Load  Data

In [7]:
df_train= pd.read_csv('../data/raw/train.csv', low_memory=False)
df_test = pd.read_csv('../data/raw/test.csv')

### 3. Data Preperation

#### [3.1] Phase 1

In [8]:
# Remove player_id and team
df_train_copy = df_train.drop(['player_id', 'team', 'type', 'conf','num','year','type'], axis =1)
df_test_copy = df_test.drop(['player_id', 'team', 'type', 'conf','num','year','type'], axis =1)

In [9]:
df_train_copy['yr'].value_counts()

yr
Jr      12401
Fr      12312
So      10979
Sr      10627
0           5
57.1        1
42.9        1
Name: count, dtype: int64

In [10]:
# Define the invalid values
invalid_values = ['0', '57.1', '42.9']

# Filter out the invalid values
df_train_copy = df_train_copy[~df_train_copy['yr'].isin(invalid_values)]

In [11]:
df_train_copy['yr'].value_counts()

yr
Jr    12401
Fr    12312
So    10979
Sr    10627
Name: count, dtype: int64

#### [3.1.1] Define some functions to convert ht to proper format

In [12]:
# It seems the height is miss input as a date where month is feet and day is inches

# Function to convert entries into dd-mmm format for uniform conversion to height later on

def convert_to_dd_mmm(entry):
    try: 
        # Handle cases with a dash and two parts (e.g., "7-Jun", "Jun-00")
        if len(entry) > 3 and '-' in entry:
            parts = entry.split('-')
            if parts[0].isdigit(): # Case like "7-Jun"
                day = parts[0]
                month = parts[1]
                return f'{day}-{month}'
            else: # Case like "Jun-00"
                month = parts[0]
                day = parts[1]
                return f'{day}-{month}'
        else:
            return None # Ignore invalid entries like "Jr", "So", "Fr", "0", etc.
            
    except Exception as e:
        return None # Handle unexpected error
            

In [13]:
# Define a function that converts the "d-mmm" date format into height
def convert_to_height(date_str):
    # Split date into day and month
    try:
        day, month = date_str.split('-')
        month_to_feet = {
            'Jan': 1,
            'Feb': 2,
            'Mar': 3,
            'Apr': 4,
            'May': 5,
            'Jun': 6,
            'Jul': 7,
            'Aug': 8,
            'Sep': 9,
            'Oct': 10,
            'Nov': 11,
            'Dec': 12,
        }
        inches = int(day)
        feet = month_to_feet.get(month, 0)
        return f'{feet}\'{inches}'
    except Exception as e:
        return None



In [14]:
# Define a function that convert height to meter
def height_to_meters(height_str):
    if height_str is not None:
        feet, inches = height_str.split("'")
        # Convert feet and inches to float
        feet = float(feet)
        inches = float(inches)
        # Convert to height
        total_inches = (feet*12) + inches
        height_in_meters = total_inches * 0.0254
    else:
        height_in_meters = 0 # Assign 0 if missing
    
    return round(height_in_meters, 2)

#### [3.1.2] Transform Dataset with the Function defined

In [15]:
# Apply function above to convert height to its proper format
df_train_copy['ht'] = df_train_copy['ht'].apply(convert_to_dd_mmm)
df_test_copy['ht'] = df_test_copy['ht'].apply(convert_to_dd_mmm)

In [16]:
df_train_copy['ht'] = df_train_copy['ht'].apply(convert_to_height)
df_test_copy['ht'] = df_test_copy['ht'].apply(convert_to_height)

In [17]:
df_train_copy['ht'] = df_train_copy['ht'].apply(height_to_meters)
df_test_copy['ht'] = df_test_copy['ht'].apply(height_to_meters)

In [18]:
# Fill missing Rec_Rank with 0
df_train_copy['Rec_Rank'] = df_train_copy['Rec_Rank'].fillna(0)
df_test_copy['Rec_Rank'] = df_test_copy['Rec_Rank'].fillna(0)

In [19]:
# Fill other missing values with the median
#Fill missing test value with median

columns_to_fill = ['rimmade', 'rimmade_rimmiss', 'midmade', 'midmade_midmiss', 'ast_tov',
                   'rim_ratio', 'mid_ratio', 'dunksmade', 'dunksmiss_dunksmade', 'dunks_ratio', 'ht', 'pick']

for column in columns_to_fill:
    df_train_copy[column] = df_train_copy[column].fillna(df_train_copy[column].median())
    df_test_copy[column] = df_test_copy[column].fillna(df_test_copy[column].median())

In [20]:
df_train_copy.shape

(46611, 58)

In [21]:
df_test_copy.shape

(4970, 57)

#### [3.2] Phase 2

In [22]:
# Drop missing values in training set
df_train_copy = df_train_copy.dropna()

In [23]:
from my_krml_ratana.data.sets import pop_target

In [24]:
# Pop target out
df_train_copy, y_train = pop_target(df_train_copy, 'drafted')

In [25]:
df_train_copy.shape

(46314, 57)

In [26]:
# Repalce missing 'yr' with Fr since we cannot drop these row of data
df_test_copy['yr'] = df_test_copy['yr'].fillna('Fr')

In [27]:
# Fill missing values with 0 in testing
df_test_copy = df_test_copy.fillna(0)

In [28]:
df_test_copy.shape

(4970, 57)

In [29]:
# Select numerical columns and categorical columns
num_col = list(df_train_copy.select_dtypes('number').columns)
cat_col = list(set(df_train_copy.columns) - set(num_col))

In [30]:
num_col

['GP',
 'Min_per',
 'Ortg',
 'usg',
 'eFG',
 'TS_per',
 'ORB_per',
 'DRB_per',
 'AST_per',
 'TO_per',
 'FTM',
 'FTA',
 'FT_per',
 'twoPM',
 'twoPA',
 'twoP_per',
 'TPM',
 'TPA',
 'TP_per',
 'blk_per',
 'stl_per',
 'ftr',
 'ht',
 'porpag',
 'adjoe',
 'pfr',
 'Rec_Rank',
 'ast_tov',
 'rimmade',
 'rimmade_rimmiss',
 'midmade',
 'midmade_midmiss',
 'rim_ratio',
 'mid_ratio',
 'dunksmade',
 'dunksmiss_dunksmade',
 'dunks_ratio',
 'pick',
 'drtg',
 'adrtg',
 'dporpag',
 'stops',
 'bpm',
 'obpm',
 'dbpm',
 'gbpm',
 'mp',
 'ogbpm',
 'dgbpm',
 'oreb',
 'dreb',
 'treb',
 'ast',
 'stl',
 'blk',
 'pts']

In [31]:
cat_col

['yr']

#### [3.2.2] One-hot Encode categorical variables

In [32]:
# One hot encode
ohe = OneHotEncoder(sparse_output=False, drop='first')

In [33]:
features_train = ohe.fit_transform(df_train_copy[cat_col])

In [34]:
features_train = pd.DataFrame(features_train, columns=ohe.get_feature_names_out())

#### [3.2.3] Scale Numerical Variables

In [35]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()


In [36]:
features_train[num_col] = scaler.fit_transform(df_train_copy[num_col])

In [37]:
# Assign X_train
X_train = features_train

In [38]:
# Apply transformation to testing dataset
X_test = ohe.fit_transform(df_test_copy[cat_col])


In [39]:
X_test = pd.DataFrame(X_test, columns=ohe.get_feature_names_out())

In [40]:
X_test

,yr_Jr,yr_So,yr_Sr
0,0.0,0.0,1.0
1,0.0,0.0,1.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0
...,...,...,...
4965,0.0,0.0,0.0
4966,0.0,0.0,0.0
4967,0.0,0.0,0.0
4968,0.0,0.0,0.0


In [41]:
X_test[num_col] = scaler.fit_transform(df_test_copy[num_col])

In [42]:
X_test

,yr_Jr,yr_So,yr_Sr,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,mp,ogbpm,dgbpm,oreb,dreb,treb,ast,stl,blk,pts
0,0.0,0.0,1.0,-1.153846,-0.551692,0.698473,-1.750000,0.121795,-0.058864,-0.616667,...,-0.318505,-0.167274,-0.644224,-0.625000,-0.625305,-0.688429,-0.529884,-0.644705,-0.399940,-0.477965
1,0.0,0.0,1.0,-0.461538,-0.277256,-1.362595,0.078947,-0.858974,-1.085180,-0.200000,...,-0.237588,-1.073519,-1.376709,-0.284125,-0.099512,-0.190571,-0.175719,-0.371239,-0.399940,-0.293640
2,0.0,0.0,1.0,-0.615385,-0.070489,-0.187023,0.763158,0.384615,0.113573,0.450000,...,0.379584,-0.249698,0.724702,1.041625,1.302617,1.256036,0.249357,0.358153,5.267327,0.588416
3,0.0,0.0,1.0,-0.769231,-0.583647,0.541985,-1.000000,2.256410,2.249307,-0.616667,...,-0.722401,0.139165,0.465755,-0.625000,-0.647627,-0.705429,-0.529884,-0.644705,-0.399940,-0.516584
4,0.0,0.0,1.0,0.461538,0.867481,0.240458,0.460526,0.378205,0.364958,-0.016667,...,0.874320,0.799251,-0.213461,0.679375,0.564106,0.590036,0.384867,0.597924,-0.138914,1.320442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4965,0.0,0.0,0.0,-1.153846,-0.602444,-1.385496,0.171053,-3.083333,-3.521468,-0.616667,...,-0.763538,0.663986,5.626622,-0.625000,-0.781608,-0.807464,-0.140263,0.107401,-0.399940,-0.613140
4966,0.0,0.0,0.0,-1.000000,-0.583647,-2.614504,-1.434211,-3.083333,-3.521468,0.550000,...,-0.645735,-1.028001,-1.575966,-0.312500,-0.781608,-0.718179,-0.529884,-0.644705,-0.399940,-0.613140
4967,0.0,0.0,0.0,-1.230769,-0.606203,-3.694656,-2.368421,-3.083333,-3.521468,-0.616667,...,-0.789717,-0.357840,-0.250416,-0.625000,-0.781608,-0.807464,-0.529884,-0.644705,-0.399940,-0.613140
4968,0.0,0.0,0.0,-1.230769,-0.606203,-3.694656,-2.368421,-3.083333,-3.521468,-0.616667,...,-0.789717,-0.060604,-0.680247,-0.625000,-0.781608,-0.807464,-0.529884,-0.644705,-0.399940,-0.613140


#### [3.2.4] Check for Missmatch between Train and Test set features

In [43]:
missing_in_test = set(features_train) - set(X_test.columns)
missing_in_train = set(X_test.columns) - set(features_train)

In [44]:
print("Missing in test set:", missing_in_test)

Missing in test set: set()


In [45]:
print("Extra in test set:", missing_in_train)

Extra in test set: set()


In [46]:
# We need to ensure the same feature dimension for train and test set
#Fill in missing feature from train with 0
for feature in features_train:
    if feature not in X_test.columns:
        X_test[feature] = 0

In [47]:
missing_in_test = set(features_train) - set(X_test.columns)
print("Missing in test set:", missing_in_test)


Missing in test set: set()


In [48]:
X_train.shape

(46314, 59)

In [49]:
X_test.shape

(4970, 59)

In [50]:
# Ensure the columns in the test set are in the same order as in the training set
X_test = X_test[features_train.columns]

In [51]:
X_train

,yr_Jr,yr_So,yr_Sr,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,mp,ogbpm,dgbpm,oreb,dreb,treb,ast,stl,blk,pts
0,0.0,1.0,0.0,-0.142857,-0.127863,0.008850,-0.208333,-0.395349,-0.525748,-0.453125,...,-0.114166,-0.223389,-0.578898,-0.399333,-0.491811,-0.511781,0.393410,-0.123203,-0.258522,-0.077942
1,0.0,1.0,0.0,0.428571,0.471374,0.495575,-0.444444,0.372093,0.308970,-0.109375,...,0.410655,0.276979,0.028613,0.138667,-0.172478,-0.109573,0.874838,-0.019586,0.278157,0.196443
2,0.0,0.0,1.0,-0.071429,0.683206,-0.039823,0.513889,-0.147287,-0.230897,-0.375000,...,0.867098,0.570354,-0.199302,0.086556,0.353071,0.252998,0.981237,0.090507,-0.363512,1.029498
3,0.0,0.0,1.0,0.142857,0.158397,0.026549,-0.291667,0.465116,0.243355,-0.062500,...,0.064587,0.223727,-0.023600,0.164778,-0.089559,-0.041409,0.323011,0.225083,0.000000,0.061972
4,0.0,0.0,1.0,0.357143,0.381679,-0.026549,0.541667,0.403101,0.294850,0.593750,...,0.325213,-0.022281,-0.122217,0.969444,0.829981,0.879036,0.131188,0.047860,0.545405,0.414526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46309,0.0,0.0,1.0,-1.928571,-0.688931,-4.296460,0.263889,-3.689922,-4.215947,-0.703125,...,-0.830111,-5.429306,-7.609785,-0.613000,-0.794472,-0.798382,-0.515979,-0.670036,-0.363512,-0.596214
46310,0.0,1.0,0.0,-1.785714,-0.685115,-3.398230,2.500000,-3.689922,-4.215947,-0.703125,...,-0.812536,-9.112194,-4.781974,-0.613000,-0.630551,-0.680115,-0.261765,-0.143579,-0.363512,-0.596214
46311,1.0,0.0,0.0,-1.928571,-0.685115,-4.296460,-2.513889,-3.689922,-4.215947,-0.703125,...,-0.724646,-0.726422,-2.047760,-0.613000,-0.794472,-0.798382,-0.515979,-0.670036,-0.363512,-0.596214
46312,0.0,1.0,0.0,-1.928571,-0.688931,-1.907080,0.263889,-3.689922,-4.215947,-0.703125,...,-0.830111,0.761555,-4.492609,-0.613000,-0.794472,-0.798382,0.246739,-0.670036,-0.363512,-0.596214


In [52]:
X_test

,yr_Jr,yr_So,yr_Sr,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,mp,ogbpm,dgbpm,oreb,dreb,treb,ast,stl,blk,pts
0,0.0,0.0,1.0,-1.153846,-0.551692,0.698473,-1.750000,0.121795,-0.058864,-0.616667,...,-0.318505,-0.167274,-0.644224,-0.625000,-0.625305,-0.688429,-0.529884,-0.644705,-0.399940,-0.477965
1,0.0,0.0,1.0,-0.461538,-0.277256,-1.362595,0.078947,-0.858974,-1.085180,-0.200000,...,-0.237588,-1.073519,-1.376709,-0.284125,-0.099512,-0.190571,-0.175719,-0.371239,-0.399940,-0.293640
2,0.0,0.0,1.0,-0.615385,-0.070489,-0.187023,0.763158,0.384615,0.113573,0.450000,...,0.379584,-0.249698,0.724702,1.041625,1.302617,1.256036,0.249357,0.358153,5.267327,0.588416
3,0.0,0.0,1.0,-0.769231,-0.583647,0.541985,-1.000000,2.256410,2.249307,-0.616667,...,-0.722401,0.139165,0.465755,-0.625000,-0.647627,-0.705429,-0.529884,-0.644705,-0.399940,-0.516584
4,0.0,0.0,1.0,0.461538,0.867481,0.240458,0.460526,0.378205,0.364958,-0.016667,...,0.874320,0.799251,-0.213461,0.679375,0.564106,0.590036,0.384867,0.597924,-0.138914,1.320442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4965,0.0,0.0,0.0,-1.153846,-0.602444,-1.385496,0.171053,-3.083333,-3.521468,-0.616667,...,-0.763538,0.663986,5.626622,-0.625000,-0.781608,-0.807464,-0.140263,0.107401,-0.399940,-0.613140
4966,0.0,0.0,0.0,-1.000000,-0.583647,-2.614504,-1.434211,-3.083333,-3.521468,0.550000,...,-0.645735,-1.028001,-1.575966,-0.312500,-0.781608,-0.718179,-0.529884,-0.644705,-0.399940,-0.613140
4967,0.0,0.0,0.0,-1.230769,-0.606203,-3.694656,-2.368421,-3.083333,-3.521468,-0.616667,...,-0.789717,-0.357840,-0.250416,-0.625000,-0.781608,-0.807464,-0.529884,-0.644705,-0.399940,-0.613140
4968,0.0,0.0,0.0,-1.230769,-0.606203,-3.694656,-2.368421,-3.083333,-3.521468,-0.616667,...,-0.789717,-0.060604,-0.680247,-0.625000,-0.781608,-0.807464,-0.529884,-0.644705,-0.399940,-0.613140


#### 4. Save sets to Model

In [53]:
X_train.to_csv('../data/processed/X_train_master.csv', index = False)

In [54]:
y_train.to_csv('../data/processed/y_train_master.csv', index = False)

In [55]:
X_test.to_csv('../data/processed/X_test_master.csv', index = False)